# Laboratorium nr 3

## Konfiguracja


Poniższe narzędzie zostało zmodyfikowane, tak aby można było z niego korzystać w nowszych IDE, które nie wspierają `%matplotlib notebook`.
Aby korzystać z `%matplotlib widget` należy doinstalować pakiet `ipympl` np. za pomocą polecenia `pip install ipympl`. Jeżeli po zainstalowaniu będą występować błędy, należy korzystać z backendu `notebook`.

Wykorzystywane biblioteki:
- matplotlib
- numpy
- ipympl

Nowe funkcje:
- Dodanie do konstruktora klasy Scene argumentu `title`, który pozwala na ustawienie tytułu danej sceny
- Dodanie do konstruktora klasy Plot argumentu `title`, który pozwala na ustawienie tytułu wykresu / okna widżetu
- Dodanie możliwości definiowania własnych zakresów osi OX oraz OY dla wykresów (argumenty klasy Plot: `xlim` oraz `ylim` jako krotka dwuelementowa definiująca przedział, gdy zostanie podany tylko zakres dla jednej osi, druga oś będzie automatycznie skalowana)

Naprawione bugi:
- Naprawiono problem, gdy po utworzeniu nowego pustego wykresu, tworzyła się kopia poprzednio utworzonego (kopiowały się sceny z ostatniego obiektu plot)
- Poprawiono metodę zamykania figur (tworzony jest okrąg o środku pierwszego punktu i promieniu adekwatnym do zakresów osi)
- Nowo dodane figury, punkty, linie są zapisywane osobno dla każdej sceny
- Zmieniono wyświetlanie się punktów i linii (punkty wyświetlają się przed liniami)


In [ ]:
# Wybór backendu matplotlib
# %matplotlib notebook
%matplotlib widget

# Narzędzie jest oparte o kilka zewnętrznych bibliotek, które potrzebujemy najpierw zaimportować.
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js

# Parametr określający jak blisko (w odsetku całego widocznego zakresu) punktu początkowego 
# wielokąta musimy kliknąć, aby go zamknąć.
TOLERANCE = 0.02

def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))

# Klasa ta trzyma obecny stan wykresu oraz posiada metody, które mają zostać wykonane
# po naciśnięciu przycisków.
class _Button_callback(object):
    def __init__(self, scenes, xlim=[], ylim=[]):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = [[] for _ in range(len(scenes))]
        self.adding_lines = False
        self.added_lines = [[] for _ in range(len(scenes))]
        self.adding_rects = False
        self.added_rects = [[] for _ in range(len(scenes))]
        # DODANO: zapisz tytuł sceny
        self.scene_title = self.scenes[0].title if len(self.scenes) and self.scenes[0].title else ""
        # DODANO: ustawienie zakresów osi wykresu
        self.xlim = xlim
        self.ylim = ylim
        self.custom_axis = len(xlim) != 0 or len(ylim) != 0

    def set_axes(self, ax):
        self.ax = ax
        
    # Metoda ta obsługuje logikę przejścia do następnej sceny.
    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        # ADDED: Change scene title
        self.scene_title = self.scenes[self.i].title
        self.draw(autoscaling = not self.custom_axis)

    # Metoda ta obsługuje logikę powrotu do poprzedniej sceny.
    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        # ADDED: Change scene title
        self.scene_title = self.scenes[self.i].title
        self.draw(autoscaling = not self.custom_axis)
        
    # Metoda ta aktywuje funkcję rysowania punktów wyłączając równocześnie rysowanie 
    # odcinków i wielokątów.
    def add_point(self, event):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points[self.i].append(PointsCollection([]))
            
    # Metoda ta aktywuje funkcję rysowania odcinków wyłączając równocześnie
    # rysowanie punktów i wielokątów.     
    def add_line(self, event):   
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines[self.i].append(LinesCollection([]))

    # Metoda ta aktywuje funkcję rysowania wielokątów wyłączając równocześnie
    # rysowanie punktów i odcinków.
    def add_rect(self, event):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()
    
    def new_rect(self):
        self.added_rects[self.i].append(LinesCollection([]))
        self.rect_points = []
        
    # Metoda ta zwraca punkty dodane w trakcie rysowania.
    def get_added_points(self):
        return self.added_points[self.i]
    
    # Metoda ta zwraca odcinki dodane w trakcie rysowania.
    def get_added_lines(self):
        return self.added_lines[self.i]
        
    # Metoda ta zwraca wielokąty dodane w trakcie rysowania.
    def get_added_figure(self):
        return self.added_rects[self.i]
  
    # Metoda ta zwraca punkty, odcinki i wielokąty dodane w trakcie rysowania
    # jako scenę.
    def get_added_elements(self):
        return Scene(self.added_points[self.i], self.added_lines[self.i]+self.added_rects[self.i])


    # Metoda odpowiedzialna za właściwą logikę rysowania nowych elementów. W
    # zależności od włączonego trybu dodaje nowe punkty, początek, koniec odcinka
    # lub poszczególne wierzchołki wielokąta. Istnieje ciekawa logika sprawdzania
    # czy dany punkt jest domykający dla danego wielokąta. Polega ona na tym, że
    # sprawdzamy czy odległość nowego punktu od początkowego jest większa od
    # średniej długości zakresu pomnożonej razy parametr TOLERANCE.   
    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[self.i][-1].add_points([new_point])
            self.draw(autoscaling = False,hand_drawing=True)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[self.i][-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling = False,hand_drawing=True)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[self.i][-1].add((self.rect_points[-1], new_point))
                self.rect_points.append(new_point)
                self.draw(autoscaling = False,hand_drawing=True)
            elif len(self.rect_points) > 1:
                # DODANO: zmiana metody wykrywania zakończenia figury.
                # Teraz punkt self.rect_points[0] jest środkiem okręgu o promieniu TOLERANCE*(mediana zakresów wykresu)
                # Gdy użytkownik kliknie w obrębie tego okręgu, figura zostanie zamknięta
                if (new_point[0] - self.rect_points[0][0]) ** 2 + (new_point[1] - self.rect_points[0][1]) ** 2 <= (np.mean([self.ax.get_xlim()[1] - self.ax.get_xlim()[0], self.ax.get_ylim()[1]-self.ax.get_ylim()[0]])*TOLERANCE) ** 2:
                    self.added_rects[self.i][-1].add((self.rect_points[-1], self.rect_points[0]))
                    self.new_rect()
                else:    
                    self.added_rects[self.i][-1].add((self.rect_points[-1], new_point))
                    self.rect_points.append(new_point)
                self.draw(autoscaling = False, hand_drawing=True)
    
    # Metoda odpowiedzialna za narysowanie całego wykresu. Warto zauważyć,
    # że zaczyna się ona od wyczyszczenia jego wcześniejszego stanu. Istnieje w
    # niej nietrywialna logika zarządzania zakresem wykresu, tak żeby, w zależności
    # od ustawionego parametru autoscaling, uniknąć sytuacji, kiedy dodawanie
    # nowych punktów przy brzegu obecnie widzianego zakresu powoduje niekorzystne
    # przeskalowanie.
    def draw(self, autoscaling = True, hand_drawing = False):
        # DODANO: ustawienie zakresów osi wykresu
        autoscale_axis='both'
        if not autoscaling:
            if self.custom_axis:
                xlim = self.xlim  
                ylim = self.ylim  

            choose_autoscale_axis = [False, False] # [OX, OY]

            if not self.custom_axis or not self.xlim:
                xlim = self.ax.get_xlim()
                choose_autoscale_axis[0] = True
            
            if not self.custom_axis or not self.ylim:
                ylim = self.ax.get_ylim()
                choose_autoscale_axis[1] = True

            if choose_autoscale_axis == [True, False]:
                autoscale_axis = 'x'
                autoscaling = not hand_drawing
            if choose_autoscale_axis == [False, True]:
                autoscale_axis = 'y'
                autoscaling = not hand_drawing

        self.ax.clear()
        for collection in (self.scenes[self.i].points + self.added_points[self.i]):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs, zorder=3)
        for collection in (self.scenes[self.i].lines + self.added_lines[self.i] + self.added_rects[self.i]):
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale(enable=autoscaling, axis=autoscale_axis)

        if autoscale_axis == 'y':
            self.ax.set_xlim(xlim)

        if autoscale_axis == 'x':
            self.ax.set_ylim(ylim)
      
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
            
        # DODANO: tytuł sceny
        fig = plt.gcf() # Pobierz aktywne okno
        fig.suptitle(self.scene_title)
        
        plt.draw()


### Interfejsy

[Dostępne kolory](https://matplotlib.org/3.1.1/gallery/color/named_colors.html)

[Dostępne znaczniki punktów](https://matplotlib.org/3.1.1/api/markers_api.html#module-matplotlib.markers)

In [ ]:
# Klasa Scene odpowiada za przechowywanie elementów, które mają być
# wyświetlane równocześnie. Konkretnie jest to lista PointsCollection i
# LinesCollection.
class Scene:
    def __init__(self, points=[], lines=[], title=""):
        self.points=points
        self.lines=lines
        # ADDED: Scene title
        self.title = '\n'+title if title != "" else ""

# Klasa PointsCollection gromadzi w sobie punkty jednego typu, a więc takie,
# które zostaną narysowane w takim samym kolorze i stylu. W konstruktorze
# przyjmuje listę punktów rozumianych jako pary współrzędnych (x, y). Parametr
# kwargs jest przekazywany do wywołania funkcji z biblioteki MatPlotLib przez
# co użytkownik może podawać wszystkie parametry tam zaproponowane.        
class PointsCollection:
    def __init__(self, points, **kwargs):
        self.points = points
        self.kwargs = kwargs
    
    def add_points(self, points):
        self.points = self.points + points

# Klasa LinesCollection podobnie jak jej punktowy odpowiednik gromadzi
# odcinki tego samego typu. Tworząc ją należy podać listę linii, gdzie każda
# z nich jest dwuelementową listą punktów – par (x, y). Parametr kwargs jest
# przekazywany do wywołania funkcji z biblioteki MatPlotLib przez co użytkownik
# może podawać wszystkie parametry tam zaproponowane.
class LinesCollection:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs
        
    def add(self, line):
        self.lines.append(line)
        
    def get_collection(self):
        return mcoll.LineCollection(self.lines, **self.kwargs)

# Klasa Plot jest najważniejszą klasą w całym programie, ponieważ agreguje
# wszystkie przygotowane sceny, odpowiada za stworzenie wykresu i przechowuje
# referencje na przyciski, dzięki czemu nie będą one skasowane podczas tzw.
# garbage collectingu.
class Plot:
    def __init__(self, scenes = [], points = [], lines = [], json = None, title="", xlim=[], ylim=[]):
        # DODANO: tytuł wykresu
        self.plot_title = title
        # DODANO: ustawienie zakresów osi wykresu
        self.xlim = xlim
        self.ylim = ylim
        self.is_not_custom_axis = len(xlim)==0 and len(ylim)==0

        # NAPRAWIONO: pobieranie scen z wcześniej utworzonego wykresu
        if not scenes:
            self.scenes = [Scene()]
        else:
            self.scenes = scenes

        if json is None:
            if points or lines:
                self.scenes[0].points = points
                self.scenes[0].lines = lines
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]], 
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]], title=scene["title"]) 
                           for scene in js.loads(json)]
    
    # Ta metoda ma szczególne znaczenie, ponieważ konfiguruje przyciski i
    # wykonuje tym samym dość skomplikowaną logikę. Zauważmy, że konfigurując każdy
    # przycisk podajemy referencję na metodę obiektu _Button_callback, która
    # zostanie wykonana w momencie naciśnięcia.
    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.2)
        ax_prev = plt.axes([0.6, 0.05, 0.15, 0.075])
        ax_next = plt.axes([0.76, 0.05, 0.15, 0.075])
        ax_add_point = plt.axes([0.44, 0.05, 0.15, 0.075])
        ax_add_line = plt.axes([0.28, 0.05, 0.15, 0.075])
        ax_add_rect = plt.axes([0.12, 0.05, 0.15, 0.075])
        b_next = Button(ax_next, 'Następny')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Poprzedni')
        b_prev.on_clicked(self.callback.prev)
        b_add_point = Button(ax_add_point, 'Dodaj punkt')
        b_add_point.on_clicked(self.callback.add_point)
        b_add_line = Button(ax_add_line, 'Dodaj linię')
        b_add_line.on_clicked(self.callback.add_line)
        b_add_rect = Button(ax_add_rect, 'Dodaj figurę')
        b_add_rect.on_clicked(self.callback.add_rect)
        return [b_prev, b_next, b_add_point, b_add_line, b_add_rect]
    
    def add_scene(self, scene):
        self.scenes.append(scene)
    
    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    # Metoda toJson() odpowiada za zapisanie stanu obiektu do ciągu znaków w
    # formacie JSON.
    def toJson(self):
        return js.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points], 
                          "lines":[linesCol.lines for linesCol in scene.lines],
                          "title": scene.title[1:]} 
                         for scene in self.scenes])    
    
    # Metoda ta zwraca punkty dodane w trakcie rysowania.
    def get_added_points(self):
        if self.callback:
            result = self.callback.get_added_points()
            return result
    
    # Metoda ta zwraca odcinki dodane w trakcie rysowania.
    def get_added_lines(self):
        if self.callback:
            result = self.callback.get_added_lines()
            return result

    # Metoda ta zwraca wielokąty dodane w trakcie rysowania.
    def get_added_figure(self):
        if self.callback:
            result = self.callback.get_added_figure()
            return result

    # Metoda ta zwraca punkty, odcinki i wielokąty dodane w trakcie rysowania
    # jako scenę.
    def get_added_elements(self):
        if self.callback:
            result = self.callback.get_added_elements()
            return result

    # Główna metoda inicjalizująca wyświetlanie wykresu.
    def draw(self):
        plt.close()
        # DODANO: tytuł wykresu
        fig = plt.figure(num=self.plot_title)
        # DODANO: ustawienie zakresów osi wykresu
        self.callback = _Button_callback(self.scenes, self.xlim, self.ylim)
        self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on = False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        # DODANO: ustawienie zakresów osi wykresu
        self.callback.draw(self.is_not_custom_axis)
        

### Przykłady użycia

##### Proste rysowanie

Rysowanie prostego rysunku złożonego ze statycznych punktów i odcinków jest stosunkowo proste. Należy utworzyć zmienną z obiektem `Plot`, który przyjmuje w konstruktorze listę obiektów `PointsCollection` jako parametr `points` oraz listę `LinesCollection` jako parametr `lines`. Następnie należy wywołać jego metodę `draw()`.

```python
plot = Plot(points=[PointsCollection([(1, 2), (3, 1.5), (2, -1)]),
               PointsCollection([(5, -2), (2, 2), (-2, -1)], color='green', marker = "^")],
            lines=[LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]])])
plot.draw()     
```

##### Rysunek dynamiczny

To narzędzie pozwala na wizualizację "rysunków dynamicznych", a więc takich, które składają się z wielu różnych obrazów. Może to być przydatne np. podczas wizualizacji kolejnych kroków danego algorytmu. Poszczególne klatki określane są jako obiekty `Scene`, które przyjmują listę `PointsCollection` oraz listę `LinesCollection` w swoim konstruktorze. Obiekt `Plot` przyjmuje w parametrze konstruktora `scenes` właśnie listę takich scen.

Na rysunku przyciski `Następny` i `Poprzedni` służą do nawigacji pomiędzy nimi.

```python
scenes=[Scene([PointsCollection([(1, 2), (3, 1.5), (2, -1)]), 
               PointsCollection([(5, -2), (2, 2), (-2, -1)], color='green', marker = "^")], 
              [LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]])], "Scene 1"), 
        Scene([PointsCollection([(1, 2), (3, 1.5), (2, -1)], color='red'), 
               PointsCollection([(5, -2), (2, 2), (-2, 1)], color='black')], 
              [LinesCollection([[(-1,2),(-2,3)], [(0,-1),(-1,0)]])],"Scene 2")]

plot=Plot(scenes = scenes)
plot.draw()
```

##### Zapis i odczyt z pliku

Klasa Plot posiada metodę `toJson()`, która zwraca string zawierający listę scen w formacie JSON. Taki string można normalnie zapisać do pliku stosując normalne sposoby dostępne w Pythonie. Wczytanie listy scen z pliku dokonuje się poprzez podanie parametru `json` w kostruktorze `Plot`.

```python
scenes=[Scene([PointsCollection([(-2, -1)], color='green', marker = "^")], 
              [LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]])]), 
        Scene([PointsCollection([(1, 2), (3, 1.5)], color='red'), 
               PointsCollection([(5, -2)], color='black')])]
plot = Plot(scenes)

with open('somefile.json', 'w') as file:
    file.write(plot.toJson())
    
#somefile.json: [{"points": [[[-2, -1]]], "lines": [[[[1, 2], [2, 3]], [[0, 1], [1, 0]]]]}, {"points": [[[1.0, 2.0], [3.0, 1.5]], [[5, -2]]], "lines": []}]
    
with open('somefile.json', 'r') as file:
    json = file.read()
    
plot2 = Plot(json=json)
plot2.draw()
```

##### Operowanie na dodanych punktach

Możemy bardzo łatwo rysować nowe punkty i linie. Aby je później wykorzystać sugerowanym sposobem jest utworzenie nowej komórki, w której utworzymy nowy `Plot` i skorzystamy z metod `get_added_points()` i `get_added_points()` lub `get_added_elements()` (zwraca scenę) z pierwszego `Plot`u.

```python 
#Dodaj tu punkty, figury i/lub odcinki!
plot1 = Plot(points=[PointsCollection([(-2,-1),(5,3)], color='red')])
plot1.draw()

plot2 = Plot([plot1.get_added_elements()])
plot2.draw()
```

## Funkcje pomocnicze

In [ ]:
from collections import deque
from copy import deepcopy

def save_plot(plot: Plot) -> None:
    filename = plot.plot_title if plot.plot_title != "" else "somefile"
    with open(f"{filename}.json", 'w') as file:
        file.write(plot.toJson())

def load_plot(filename: str) -> Plot:
    try:
        with open(f"{filename}.json", 'r') as file:
            json = file.read()
        print(json)
        plot = Plot(json=json)
        return plot, True
    except FileNotFoundError:
        return None, False

def det3x3(a: tuple[float, float],b: tuple[float, float],c: tuple[float, float]) -> float:
    matrix = [[],[],[]]
    matrix[0] = [a[0], a[1], 1]
    matrix[1] = [b[0], b[1], 1]
    matrix[2] = [c[0], c[1], 1]

    f1 = matrix[0][0] * matrix[1][1] * matrix[2][2]
    f2 = matrix[0][1] * matrix[1][2] * matrix[2][0]
    f3 = matrix[1][0] * matrix[2][1] * matrix[0][2]
    f4 = matrix[0][2] * matrix[1][1] * matrix[2][0]
    f5 = matrix[1][2] * matrix[2][1] * matrix[0][0]
    f6 = matrix[1][0] * matrix[0][1] * matrix[2][2]
    return f1+f2+f3-f4-f5-f6
    
def point_position(a: tuple[float, float],b: tuple[float, float], c: tuple[float,float], epsilon: float = 1e-10) -> int:
    """ Retruns position of point c relative to vector ab.

    :param tuple[float, float] a: first coordinate of vector ab  
    :param tuple[float, float] b: second coordinate of vector ab  
    :param tuple[float, float] c: tested point
    :param float epsilon: zero tolerance for determinant
    :return int: position of tested point as value of a dict: {'left' : 1, 'right': -1, 'collinear': 0}
    """

    determinant = det3x3(a,b,c)

    if determinant < -epsilon:
        return -1
    elif determinant <= epsilon:
        return 0
    else:
        return 1


class Polygon:
    """Class representing polygons. Uses set data structure for faster finding duplicates.
    """
    def __init__(self, lines: list[LinesCollection]=[]):
        self.lines = []
        self.points = []
        self.lines_set = set()
        self.points_set = set()
        self.points_index = {}
        self.points_dict = {}
        self.added_diagonals = []

        if len(lines) > 0:
            self.read_lines(lines[0])

    def save_triangulation(self, filename="triangulation"):
        if len(self.points) > 0 and len(self.added_diagonals) > 0 and len(self.lines) > 0:
            indexed_lines = set()
            for line in self.lines:
                a1 = self.points.index(line[0])
                a2 = self.points.index(line[1])
                indexed_lines.add((min(a1,a2), max(a1,a2)))
            for line in self.added_diagonals:
                a1 = self.points.index(line[0])
                a2 = self.points.index(line[1])
                indexed_lines.add((min(a1,a2), max(a1,a2)))

            indexed_lines = list(indexed_lines)
            
            # Save to file 
            serialized_json = js.dumps({"points": self.points, "lines": indexed_lines})   
            with open(f"{filename}.json",'w') as f:
                f.write(serialized_json)
                
        else:
            print("Nie striangulowano wielokąta!")

    def read_lines(self, lines: LinesCollection):
        for line in lines.lines:
            self.add_line(((line[0][0], line[0][1]),(line[1][0],line[1][1])))

    def add_line(self, line: tuple[tuple[float,float], tuple[float,float]]) -> bool:
        if line in self.lines_set:
            return False

        # Check if points are neighbours
        if line[0] in self.points_index and line[1] in self.points_index:
            i1 = self.points_index[line[0]]
            i2 = self.points_index[line[1]]
            if abs(i1-i2) == 1 or (i1 == len(self.points)-1 and i2==0) or (i2 == len(self.points)-1 and i1 == 0):
                return False

        self.lines.append(line)
        self.lines_set.add(line)
        return True

    def add_point(self, point: tuple[float,float], index, color="black") -> bool:
        if point in self.points_set:
            return False
        self.points.append(point)
        self.points_set.add(point)
        self.points_index[point] = index
        
        if color not in self.points_dict:
            self.points_dict[color] = PointsCollection([point], color=color)
        else:
            self.points_dict[color].points.append(point)
        
        return True

    def get_lines(self, copy=False):
        if copy:
            return LinesCollection(self.lines[:], linewidths=0.5, color="black")
        return LinesCollection(self.lines, linewidths=0.5, color="black")

    def get_points(self):
        return [self.points_dict[color] for color in self.points_dict]


## Algorytm wykrywania y-monotonicznej figury

In [ ]:
def is_polygon_y_monotonic(polygon: Polygon) -> bool:
    """ Checks if polygon is y-monotonic.

    :param LinesCollection polygon: List of lines defined anticlockwise as LinesCollection object
    :return bool: result 
    """

    lines = polygon.lines
    n = len(lines)
    for i in range(n):
        p1,p2 = lines[(i-1)%n]
        p2,p3 = lines[i%n]

        # Merge vertex
        if p1[1] > p2[1] and p3[1] > p2[1] and point_position(p1,p2,p3) == -1:
            return False

        # Split vertex
        if p1[1] < p2[1] and p3[1] < p2[1] and point_position(p1,p2,p3) == -1:
            return False

    return True

## Algorytm wykrywający punkty początkowe, końcowe, łączące, dzielące, prawidłowe w wielokącie

Klasyfikacja punktów:
- Kolor zielony: początkowy: obaj sąsiedzi leżą poniżej oraz kąt wewnętrzny jest mniejszy od $\pi$
- Kolor czerwony: końcowy: obaj sąsiedzi leżą powyżej oraz kąt wewnętrzny jest mniejszy od $\pi$
- Kolor niebieski: łączący: obaj sąsiedzi leżą powyżej oraz kąt wewnętrzny jest większy od $\pi$
- Kolor cyjan: dzielący: obaj sąsiedzi leżą poniżej oraz kąt wewnętrzny jest większy od $\pi$
- Kolor brązowy: prawidłowy: w pozostałych przypadkach (jeden punkt leży powyżej, drugi poniżej)

In [ ]:
def detect_points_in_polygon(polygon: Polygon) -> None:
    # start_vertex = PointsCollection([], color="tab:green")
    # end_vertex = PointsCollection([], color="tab:red")
    # merge_vertex = PointsCollection([], color="darkblue")
    # split_vertex = PointsCollection([], color="tab:cyan")
    # regular_vertex = PointsCollection([], color="tab:brown")
    
    lines = polygon.lines
    n = len(lines)
    for i in range(n):
        p1,p2 = lines[(i-1)%n]
        p2,p3 = lines[i%n]

        # Start vertex
        if p1[1] < p2[1] and p3[1] < p2[1] and point_position(p1,p2,p3) == 1:
            polygon.add_point((p2[0], p2[1]),i, "tab:green")

        # End vertex
        elif p1[1] > p2[1] and p3[1] > p2[1] and point_position(p1,p2,p3) == 1:
            polygon.add_point((p2[0], p2[1]),i, "tab:red")

        # Merge vertex
        elif p1[1] > p2[1] and p3[1] > p2[1] and point_position(p1,p2,p3) == -1:
            polygon.add_point((p2[0], p2[1]),i, "darkblue")

        # Split vertex
        elif p1[1] < p2[1] and p3[1] < p2[1] and point_position(p1,p2,p3) == -1:
            polygon.add_point((p2[0], p2[1]),i, "tab:cyan")

        # Regular vertex
        else:
            polygon.add_point((p2[0], p2[1]),i, "tab:brown")
            

## Algorytm trianguryzacji wielokąta y-monotonicznego na wielokąty monotoniczne


In [ ]:
def triangulize(polygon: Polygon):
    # Polygon cannot have points on a same y coordinate with this approach
    
    # Check if polygon is monotonic
    if not is_polygon_y_monotonic(polygon):
        print("Figura nie jest y-monotoniczna!")
        return 

    points = polygon.points
    # Find chains
    left_chain, right_chain = set(), set()
    start_vertex = max(points, key=lambda x: (x[1],x[0]))
    end_vertex = min(points, key=lambda x: (x[1],x[0]))

    i = 0
    n = len(points)
    p1 = points[0]
    # Find start vertex
    while p1[0] != start_vertex[0] and p1[1] != start_vertex[1]:
        i  = (i+1)%n
        p1 = points[i]

    # Add to left chain
    while p1[0] != end_vertex[0] and p1[1] != end_vertex[1]:
        left_chain.add(p1)
        i  = (i+1)%n
        p1 = points[i]

    # Add to right chain
    while p1[0] != start_vertex[0] and p1[1] != start_vertex[1]:
        right_chain.add(p1)
        i  = (i+1)%n
        p1 = points[i]


    # Sort points (decreasing y-coordinate)
    sorted_points = sorted(points, key=lambda x:(-x[1]))

    # Add vertices to stack
    s = deque()
    s.append(sorted_points[0])
    s.append(sorted_points[1])


    def is_in_triangle(a,b,c, epsilon=1e-10):
        """Why does it work?
        Let a = stack[-2], b = stack[-1].
        We know that points are sorted by y-coord descending, so a[1] < b[1]. Let's create line from a to b. Now we have to detect where we have possibility to create triangle, if next vertex has y-coord lower than a point. If b is on left chain, then point has to be on a left of ab segment. If b is on right chain, point has to be on the right of ab segment.
        """
        nonlocal left_chain

        if b in left_chain:
            return det3x3((a[0],a[1]), (b[0],b[1]), (c[0],c[1])) > epsilon
        return det3x3((a[0],a[1]), (b[0],b[1]), (c[0],c[1])) < -epsilon
        
    # Scene configuration
    scene_container = []
    added_lines = []

    for i in range(2, n-1):

        scene_container.append(Scene(title="Szukanie nowej przekątnej",points=[PointsCollection(points=polygon.points, color="black"), PointsCollection(points=list(s)[:], color="green"), PointsCollection(points=[sorted_points[i]], color="red")], lines=[LinesCollection(lines=polygon.lines[:], **polygon.get_lines().kwargs), LinesCollection(added_lines[:])]))

        # In what chain is vertex on top of stack
        chain_s_top = left_chain if s[-1] in left_chain else right_chain
        if sorted_points[i] not in chain_s_top:
            # Add diagonal for every popped vertex (except the last one)
            while len(s) > 0:
                u = s.pop()
                if len(s) > 0:
                    was_added = polygon.add_line(((u[0], u[1]), (sorted_points[i][0],sorted_points[i][1])))

                    if was_added:
                        added_lines.append(((u[0], u[1]), (sorted_points[i][0],sorted_points[i][1])))
                        scene_container.append(Scene(title="Dodawanie nowej przekątnej",points=[PointsCollection(points=polygon.points, color="black")], lines=[LinesCollection(lines=polygon.lines[:], **polygon.get_lines().kwargs), LinesCollection(added_lines[:])]))

            s.append(sorted_points[i-1])
            s.append(sorted_points[i])
        else:
            u = s.pop()
            # Pop the other vertices from s as long as the diagonals are inside polygon.
            while len(s) > 0 and is_in_triangle(s[-1], u, sorted_points[i]):
                # Take top point from stack, then pop it
                u = s.pop()
     
                was_added = polygon.add_line(((u[0], u[1]), (sorted_points[i][0],sorted_points[i][1])))

                if was_added:
                    added_lines.append(((u[0], u[1]), (sorted_points[i][0],sorted_points[i][1])))
                    scene_container.append(Scene(title="Dodawanie nowej przekątnej",points=[PointsCollection(points=polygon.points, color="black")], lines=[LinesCollection(lines=polygon.lines[:], **polygon.get_lines().kwargs), LinesCollection(added_lines[:])]))


            s.append(u)
            s.append(sorted_points[i])

    # Connect u_n to all vertices in S (except first and last)
    s.pop()
    s.popleft()
    while len(s) > 0:
        u = s.pop()
        was_added = polygon.add_line(((u[0], u[1]), (sorted_points[-1][0],sorted_points[-1][1])))

        if was_added:
            added_lines.append(((u[0], u[1]), (sorted_points[-1][0],sorted_points[-1][1])))
            scene_container.append(Scene(title="Dodawanie nowej przekątnej",points=[PointsCollection(points=polygon.points, color="black")], lines=[LinesCollection(lines=polygon.lines[:], **polygon.get_lines().kwargs), LinesCollection(added_lines[:])]))


    # Add result 
    scene_container.insert(0, Scene(points=polygon.get_points(), lines=[polygon.get_lines()], title="Rezultat triangulacji wielokąta"))

    polygon.added_diagonals = added_lines[:]
    return scene_container


## Generacja testowych wielokątów

In [ ]:
plot1, result = load_plot('polygon1')
if not result:
    # Draw polygon
    plot1 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 1")])
plot1.plot_title = "polygon1"
plot1.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot1
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot2, result = load_plot('polygon2')
if not result:
    # Draw polygon
    plot2 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 2")])
plot2.plot_title = "polygon2"
plot2.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot2
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot3, result = load_plot('polygon3')
if not result:
    # Draw polygon
    plot3 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 3")])
plot3.plot_title = "polygon3"
plot3.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot3
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot4, result = load_plot('polygon4')
if not result:
    # Draw polygon
    plot4 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 4")])
plot4.plot_title = "polygon4"
plot4.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot4
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot5, result = load_plot('polygon5')
if not result:
    # Draw polygon
    plot5 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 5")])
plot5.plot_title = "polygon5"
plot5.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot5
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot6, result = load_plot('polygon6')
if not result:
    # Draw polygon
    plot6 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 6")])
plot6.plot_title = "polygon6"
plot6.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot6
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot7, result = load_plot('polygon7')
if not result:
    # Draw polygon
    plot7 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 7")])
plot7.plot_title = "polygon7"
plot7.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot7
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot8, result = load_plot('polygon8')
if not result:
    # Draw polygon
    plot8 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 8")])
plot8.plot_title = "polygon8"
plot8.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot8
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot9, result = load_plot('polygon9')
if not result:
    # Draw polygon
    plot9 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 9")])
plot9.plot_title = "polygon9"
plot9.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot9
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot10, result = load_plot('polygon10')
if not result:
    # Draw polygon
    plot10 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 10")])
plot10.plot_title = "polygon10"
plot10.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot10
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot11, result = load_plot('polygon11')
if not result:
    # Draw polygon
    plot11 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 11")])
plot11.plot_title = "polygon11"
plot11.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot11
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot12, result = load_plot('polygon12')
if not result:
    # Draw polygon
    plot12 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 12")])
plot12.plot_title = "polygon12"
plot12.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot12
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot13, result = load_plot('polygon13')
if not result:
    # Draw polygon
    plot13 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 13")])
plot13.plot_title = "polygon13"
plot13.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot13
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
plot14, result = load_plot('polygon14')
if not result:
    # Draw polygon
    plot14 = Plot(xlim=[0,15], ylim=[0,15],scenes=[Scene(title="Figura nr 14")])
plot14.plot_title = "polygon14"
plot14.draw()

In [ ]:
# Run cell to save polygon
plot_to_save = plot14
save_plot(Plot(scenes=[Scene(lines=plot_to_save.get_added_figure(), title=plot_to_save.scenes[0].title[1:])], title=plot_to_save.plot_title))

In [ ]:
polygon1 = Polygon(plot1.get_added_figure() if len(plot1.get_added_figure()) != 0 else plot1.scenes[0].lines)
polygon2 = Polygon(plot2.get_added_figure() if len(plot2.get_added_figure()) != 0 else plot2.scenes[0].lines)
polygon3 = Polygon(plot3.get_added_figure() if len(plot3.get_added_figure()) != 0 else plot3.scenes[0].lines)
polygon4 = Polygon(plot4.get_added_figure() if len(plot4.get_added_figure()) != 0 else plot4.scenes[0].lines)
polygon5 = Polygon(plot5.get_added_figure() if len(plot5.get_added_figure()) != 0 else plot5.scenes[0].lines)
polygon6 = Polygon(plot6.get_added_figure() if len(plot6.get_added_figure()) != 0 else plot6.scenes[0].lines)
polygon7 = Polygon(plot7.get_added_figure() if len(plot7.get_added_figure()) != 0 else plot7.scenes[0].lines)
polygon8 = Polygon(plot8.get_added_figure() if len(plot8.get_added_figure()) != 0 else plot8.scenes[0].lines)
polygon9 = Polygon(plot9.get_added_figure() if len(plot9.get_added_figure()) != 0 else plot9.scenes[0].lines)
polygon10 = Polygon(plot10.get_added_figure() if len(plot10.get_added_figure()) != 0 else plot10.scenes[0].lines)
polygon11 = Polygon(plot11.get_added_figure() if len(plot11.get_added_figure()) != 0 else plot11.scenes[0].lines)
polygon12 = Polygon(plot12.get_added_figure() if len(plot12.get_added_figure()) != 0 else plot12.scenes[0].lines)
polygon13 = Polygon(plot13.get_added_figure() if len(plot13.get_added_figure()) != 0 else plot13.scenes[0].lines)
polygon14 = Polygon(plot14.get_added_figure() if len(plot14.get_added_figure()) != 0 else plot14.scenes[0].lines)


## Testowanie wielokątów, czy są monotoniczne

In [ ]:
print(f"Y-monotoniczność figury 1: {is_polygon_y_monotonic(polygon1)}")
print(f"Y-monotoniczność figury 2: {is_polygon_y_monotonic(polygon2)}")
print(f"Y-monotoniczność figury 3: {is_polygon_y_monotonic(polygon3)}")
print(f"Y-monotoniczność figury 4: {is_polygon_y_monotonic(polygon4)}")
print(f"Y-monotoniczność figury 5: {is_polygon_y_monotonic(polygon5)}")
print(f"Y-monotoniczność figury 6: {is_polygon_y_monotonic(polygon6)}")
print(f"Y-monotoniczność figury 7: {is_polygon_y_monotonic(polygon7)}")
print(f"Y-monotoniczność figury 8: {is_polygon_y_monotonic(polygon8)}")
print(f"Y-monotoniczność figury 9: {is_polygon_y_monotonic(polygon9)}")
print(f"Y-monotoniczność figury 10: {is_polygon_y_monotonic(polygon10)}")
print(f"Y-monotoniczność figury 11: {is_polygon_y_monotonic(polygon11)}")
print(f"Y-monotoniczność figury 12: {is_polygon_y_monotonic(polygon12)}")
print(f"Y-monotoniczność figury 13: {is_polygon_y_monotonic(polygon13)}")
print(f"Y-monotoniczność figury 14: {is_polygon_y_monotonic(polygon14)}")


## Klasyfikacja punktów w wielokącie

In [ ]:
detect_points_in_polygon(polygon1)
tmp_polygon = polygon1
plot = Plot(title="polygon1_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 1",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon2)
tmp_polygon = polygon2
plot = Plot(title="polygon2_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 2",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon3)
tmp_polygon = polygon3
plot = Plot(title="polygon3_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 3",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon4)
tmp_polygon = polygon4
plot = Plot(title="polygon4_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 4",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon5)
tmp_polygon = polygon5
plot = Plot(title="polygon5_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 5",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon6)
tmp_polygon = polygon6
plot = Plot(title="polygon6_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 6",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon7)
tmp_polygon = polygon7
plot = Plot(title="polygon7_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 7",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon8)
tmp_polygon = polygon8
plot = Plot(title="polygon8_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 8",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon9)
tmp_polygon = polygon9
plot = Plot(title="polygon9_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 9",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon10)
tmp_polygon = polygon10
plot = Plot(title="polygon10_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 10",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon11)
tmp_polygon = polygon11
plot = Plot(title="polygon11_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 11",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon12)
tmp_polygon = polygon12
plot = Plot(title="polygon12_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 12",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon13)
tmp_polygon = polygon13
plot = Plot(title="polygon13_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 13",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

In [ ]:
detect_points_in_polygon(polygon14)
tmp_polygon = polygon14
plot = Plot(title="polygon14_classification", scenes=[Scene(title="Klasyfikacja punktów w figurze nr 14",points=tmp_polygon.get_points(), lines=[tmp_polygon.get_lines()])])
plot.draw()

## Triangulacja wielokątów y-monotonicznych

In [ ]:
polygon1_copy = deepcopy(polygon1)
scenes = triangulize(polygon1_copy)
plot = Plot(title="polygon1_triangulize", scenes=scenes)
plot.draw()
polygon1.added_diagonals = polygon1_copy.added_diagonals[:]

In [ ]:
polygon1.save_triangulation('polygon1_triangulation')

In [ ]:
polygon2_copy = deepcopy(polygon2)
scenes = triangulize(polygon2_copy)
plot = Plot(title="polygon2_triangulize", scenes=scenes)
plot.draw()
polygon2.added_diagonals = polygon2_copy.added_diagonals[:]

In [ ]:
polygon2.save_triangulation('polygon2_triangulation')

In [ ]:
polygon3_copy = deepcopy(polygon3)
scenes = triangulize(polygon3_copy)
plot = Plot(title="polygon3_triangulize", scenes=scenes)
plot.draw()
polygon3.added_diagonals = polygon3_copy.added_diagonals[:]

In [ ]:
polygon3.save_triangulation('polygon3_triangulation')

In [ ]:
polygon4_copy = deepcopy(polygon4)
scenes = triangulize(polygon4_copy)
plot = Plot(title="polygon4_triangulize", scenes=scenes)
plot.draw()
polygon4.added_diagonals = polygon4_copy.added_diagonals[:]

In [ ]:
polygon4.save_triangulation('polygon4_triangulation')

In [ ]:
polygon5_copy = deepcopy(polygon5)
scenes = triangulize(polygon5_copy)
plot = Plot(title="polygon5_triangulize", scenes=scenes)
plot.draw()
polygon5.added_diagonals = polygon5_copy.added_diagonals[:]

In [ ]:
polygon5.save_triangulation('polygon5_triangulation')

In [ ]:
polygon6_copy = deepcopy(polygon6)
scenes = triangulize(polygon6_copy)
plot = Plot(title="polygon6_triangulize", scenes=scenes)
plot.draw()
polygon6.added_diagonals = polygon6_copy.added_diagonals[:]

In [ ]:
polygon6.save_triangulation('polygon6_triangulation')

In [ ]:
polygon7_copy = deepcopy(polygon7)
scenes = triangulize(polygon7_copy)
plot = Plot(title="polygon7_triangulize", scenes=scenes)
plot.draw()
polygon7.added_diagonals = polygon7_copy.added_diagonals[:]

In [ ]:
polygon7.save_triangulation('polygon7_triangulation')

In [ ]:
polygon8_copy = deepcopy(polygon8)
scenes = triangulize(polygon8_copy)
plot = Plot(title="polygon8_triangulize", scenes=scenes)
plot.draw()
polygon8.added_diagonals = polygon8_copy.added_diagonals[:]

In [ ]:
polygon8.save_triangulation('polygon8_triangulation')

In [ ]:
polygon9_copy = deepcopy(polygon9)
scenes = triangulize(polygon9_copy)
plot = Plot(title="polygon9_triangulize", scenes=scenes)
plot.draw()
polygon9.added_diagonals = polygon9_copy.added_diagonals[:]

In [ ]:
polygon9.save_triangulation('polygon9_triangulation')

In [ ]:
polygon10_copy = deepcopy(polygon10)
scenes = triangulize(polygon10_copy)
plot = Plot(title="polygon10_triangulize", scenes=scenes)
plot.draw()
polygon10.added_diagonals = polygon10_copy.added_diagonals[:]

In [ ]:
polygon10.save_triangulation('polygon10_triangulation')

In [ ]:
polygon11_copy = deepcopy(polygon11)
scenes = triangulize(polygon11_copy)
plot = Plot(title="polygon11_triangulize", scenes=scenes)
plot.draw()
polygon11.added_diagonals = polygon11_copy.added_diagonals[:]

In [ ]:
polygon11.save_triangulation('polygon11_triangulation')

In [ ]:
polygon12_copy = deepcopy(polygon12)
scenes = triangulize(polygon12_copy)
plot = Plot(title="polygon12_triangulize", scenes=scenes)
plot.draw()
polygon12.added_diagonals = polygon12_copy.added_diagonals[:]

In [ ]:
polygon12.save_triangulation('polygon12_triangulation')

In [ ]:
polygon13_copy = deepcopy(polygon13)
scenes = triangulize(polygon13_copy)
plot = Plot(title="polygon13_triangulize", scenes=scenes)
plot.draw()
polygon13.added_diagonals = polygon13_copy.added_diagonals[:]

In [ ]:
polygon13.save_triangulation('polygon13_triangulation')

In [ ]:
polygon14_copy = deepcopy(polygon14)
scenes = triangulize(polygon14_copy)
plot = Plot(title="polygon14_triangulize", scenes=scenes)
plot.draw()
polygon14.added_diagonals = polygon14_copy.added_diagonals[:]

In [ ]:
polygon14.save_triangulation('polygon14_triangulation')